This recommender is built using content from [Amazon Product Dataset 2020](https://data.world/promptcloud/amazon-product-dataset-2020/workspace/file?filename=marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv%2Fhome%2Fsdf%2Fmarketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv) and [Amazon Co-Ecommerce Sample](https://data.world/promptcloud/fashion-products-on-amazon-com/workspace/file?filename=amazon_co-ecommerce_sample.csv).

Make sure Rake (Rapid Automatic Keyword Extraction algorithm) library is installed. If not, it can be installed via "pip install rake_nltk". Refer to https://pypi.org/project/rake-nltk/ for more information.

This python script is adapted from a Digital Business Technology and Transformation module I have taken in school.

In [1]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

### Importing Data

Read in and analyse input data

In [2]:
df = pd.read_csv('marketing_sample_amazon_ecommerce_home_sdf_20200101_20200131_10k_data.csv')
df.head(10)

,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,...,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
0,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",NaN,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,NaN,...,https://www.amazon.com/DB-Longboards-CoreFlex-...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",NaN,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,55324,...,https://www.amazon.com/Electronic-Circuits-Cla...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,NaN,...,https://www.amazon.com/3Doodler-Plastic-Innova...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
3,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,142,...,https://www.amazon.com/Guillow-Airplane-Design...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
4,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,NaN,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,62151,...,https://www.amazon.com/Woodstock-Collage-500-p...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
5,f8c32a45e507a177992973cf0d46d20c,"Terra by Battat – 4 Dinosaur Toys, Medium – Di...",NaN,NaN,NaN,NaN,NaN,$18.66,NaN,AN4054Z,...,https://www.amazon.com/Terra-Battat-Dinosaurs-...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
6,40d3cd16b41970ae6872e914aecf2c8e,Rubie's Child's Pokemon Deluxe Pikachu Costume...,NaN,NaN,"Clothing, Shoes & Jewelry | Costumes & Accesso...",NaN,NaN,$29.12,NaN,B07BZJM592,...,https://www.amazon.com/Rubies-Deluxe-Pikachu-P...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
7,bc178f33a04dbccefa95b165f8b56830,Hoffmaster 120813 Double-Tipped Triangular Cra...,NaN,NaN,Toys & Games | Arts & Crafts | Drawing & Paint...,NaN,NaN,$97.68,NaN,NaN,...,https://www.amazon.com/Hoffmaster-120813-Doubl...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
8,cc2083338a16c3fe2f7895289d2e98fe,"ARTSCAPE Etched Glass 24"" x 36"" Window Film, 2...",NaN,NaN,Home & Kitchen | Home Décor | Window Treatment...,NaN,NaN,$12.99,NaN,01-0121,...,https://www.amazon.com/ARTSCAPE-Etched-Glass-W...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
9,69828c5570dcdbd1609216d33fbad0db,Pokemon TCG: Sun and Moon Crimson Invasion Eli...,NaN,NaN,NaN,NaN,NaN,$38.49,NaN,820650802607-1,...,https://www.amazon.com/Pokemon-TCG-Crimson-Inv...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


In [3]:
df.shape

(10002, 28)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 28 columns):
Uniq Id                  10002 non-null object
Product Name             10002 non-null object
Brand Name               0 non-null float64
Asin                     0 non-null float64
Category                 9172 non-null object
Upc Ean Code             34 non-null object
List Price               0 non-null float64
Selling Price            9895 non-null object
Quantity                 0 non-null float64
Model Number             8232 non-null object
About Product            9729 non-null object
Product Specification    8370 non-null object
Technical Details        9212 non-null object
Shipping Weight          8864 non-null object
Product Dimensions       479 non-null object
Image                    10002 non-null object
Variants                 2478 non-null object
Sku                      0 non-null float64
Product Url              10002 non-null object
Stock                    0

### Synthetic Values

1.) Randomly generating User ID from a range of 1 to 2000 (inclusive)

In [5]:
np.random.seed(100)
new_col = list(np.random.randint(1, 2001, df.shape[0]))
df.insert(loc=0, column='User ID', value=new_col)
df.head()

,User ID,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,...,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
0,1545,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",NaN,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,...,https://www.amazon.com/DB-Longboards-CoreFlex-...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,793,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",NaN,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,...,https://www.amazon.com/Electronic-Circuits-Cla...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,1860,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,...,https://www.amazon.com/3Doodler-Plastic-Innova...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
3,1896,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,...,https://www.amazon.com/Guillow-Airplane-Design...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
4,1880,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,NaN,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,...,https://www.amazon.com/Woodstock-Collage-500-p...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


2.) Take product description and product information from another csv as it was missing in the first

In [6]:
df_desc = pd.read_csv('amazon_co-ecommerce_sample.csv')
df_desc.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [7]:
df.drop(columns = ['Product Description'], inplace = True)
new_df = pd.concat([df, df_desc], axis=1)

# replace punctuation
new_df['Product Name'] = new_df['Product Name'].str.replace('[^\w\s]','')
new_df['product_description'] = new_df['product_description'].str.replace(r"\(.*?\):","")
new_df['product_information'] = new_df['product_information'].str.replace(r"\(.*?\):","")

new_df.head()

,User ID,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,...,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,1545,4c69b61db1fc16e7013b43fc926e502d,DB Longboards CoreFlex Crossbow 41 Bamboo Fibe...,NaN,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,...,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,793,66d49bbed043f5be260fa9f7fbff5957,Electronic Snap Circuits Mini Kits Classpack F...,NaN,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,...,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,1860,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,...,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,1896,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,...,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,1880,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock Collage 500 pc Puzzle,NaN,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,...,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [8]:
df.to_csv("combined_amazon.csv")

Use the following input features to base the recommendations.

In [9]:
df = new_df[['User ID', 'Uniq Id', 'Product Name', 'Category', 'product_information', 'product_description']].copy()
df.head()

,User ID,Uniq Id,Product Name,Category,product_information,product_description
0,1545,4c69b61db1fc16e7013b43fc926e502d,DB Longboards CoreFlex Crossbow 41 Bamboo Fibe...,Sports & Outdoors | Outdoor Recreation | Skate...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...
1,793,66d49bbed043f5be260fa9f7fbff5957,Electronic Snap Circuits Mini Kits Classpack F...,Toys & Games | Learning & Education | Science ...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...
2,1860,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,Toys & Games | Arts & Crafts | Craft Kits,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...
3,1896,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,Toys & Games | Hobbies | Models & Model Kits |...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...
4,1880,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock Collage 500 pc Puzzle,Toys & Games | Puzzles | Jigsaw Puzzles,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...


In [10]:
df.dropna(inplace=True)

**Data pre-processing** Part 1: Transforming product name, description, info and categories in single words so they are considered as unique values.

In [11]:
# discarding the delimeters between the category col
df['Category'] = df['Category'].map(lambda x: x.split(' | '))

Part  2: Extracting the key words from the product description and information

In [12]:
# initializing the new column
def extract_keywords(new_col, old_col):
    megalist = list()
    for index, row in df.iterrows():
        content = row[old_col]
        
        # instantiating Rake, by default is uses english stopwords from NLTK
        # and discard all puntuation characters
        r = Rake()

        # extracting the words by passing the text
        r.extract_keywords_from_text(content)

        # getting the dictionary whith key words and their scores
        key_words_dict_scores = r.get_word_degrees()

        # assigning the key words to the new column
        megalist.append(list(key_words_dict_scores.keys()))

    # dropping the old column
    # new_col will now replace old_col
    df.drop(columns = [old_col], inplace = True)
    df[new_col] = megalist

extract_keywords("Key_words_desc", "product_description")
extract_keywords("Key_words_info", "product_information")

In [13]:
df.head()

,User ID,Uniq Id,Product Name,Category,Key_words_desc,Key_words_info
0,1545,4c69b61db1fc16e7013b43fc926e502d,DB Longboards CoreFlex Crossbow 41 Bamboo Fibe...,"[Sports & Outdoors, Outdoor Recreation, Skates...","[product, description, hornby, 2014, catalogue...","[date, first, available24, dec, update, produc..."
1,793,66d49bbed043f5be260fa9f7fbff5957,Electronic Snap Circuits Mini Kits Classpack F...,"[Toys & Games, Learning & Education, Science K...","[si, smoke, size, name, toy, light, battery, o...","[railway, sets, ty1017, 261, toys, item, model..."
2,1860,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,"[Toys & Games, Arts & Crafts, Craft Kits]","[track, 104, cm, width, layouts, length, big, ...","[update, product, info, railway, sets, 72, giv..."
3,1896,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,"[Toys & Games, Hobbies, Models & Model Kits, M...","[r4410a, hornby, 00, gauge, br, hawksworth, 3r...","[update, product, info, 6, x, 9, railway, sets..."
4,1880,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock Collage 500 pc Puzzle,"[Toys & Games, Puzzles, Jigsaw Puzzles]","[china, functional, sharp, edges, choking, haz...","[256, update, product, info, railway, sets, 4,..."


#### Bag of words

In [14]:
bowlist = list()

columns = df.columns[2:]
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Product Name':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    bowlist.append(words)
    
df["bag_of_words"] = bowlist
df_final = df[['User ID', 'Uniq Id','bag_of_words']]
df_final.head()

,User ID,Uniq Id,bag_of_words
0,1545,4c69b61db1fc16e7013b43fc926e502d,DB Longboards CoreFlex Crossbow 41 Bamboo Fibe...
1,793,66d49bbed043f5be260fa9f7fbff5957,Electronic Snap Circuits Mini Kits Classpack F...
2,1860,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...
3,1896,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...
4,1880,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock Collage 500 pc Puzzle Toys & Games P...


### Recommendations Algorithm

1.) Create the model using Count Matrix &  Cosine Similarity Matrix

In [15]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df_final['bag_of_words'])

In [16]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.3379723 , 0.43825691, ..., 0.43676087, 0.37155086,
        0.34027533],
       [0.3379723 , 1.        , 0.4921155 , ..., 0.35433973, 0.4042782 ,
        0.33270022],
       [0.43825691, 0.4921155 , 1.        , ..., 0.44835165, 0.41874069,
        0.40024219],
       ...,
       [0.43676087, 0.35433973, 0.44835165, ..., 1.        , 0.39183588,
        0.44223199],
       [0.37155086, 0.4042782 , 0.41874069, ..., 0.39183588, 1.        ,
        0.44204102],
       [0.34027533, 0.33270022, 0.40024219, ..., 0.44223199, 0.44204102,
        1.        ]])

2.) Function that takes in either a user ID or the Product Name as input and returns the top 10 recommended products

Test and run the model (recommender)

In [17]:
def recommendations(target, cosine_sim = cosine_sim):
    
    recommended_products = []
    
    # creating a Series for the user ID so they are associated to an ordered numerical
    # list I will use later to match the indexes
    if isinstance(target, int): # User ID
        indices = pd.Series(df_final['User ID'])
        indices[:5]
    else:
        indices = pd.Series(df['Product Name'])
        indices[:5]
    
#     print(indices)
        
    
    # getting ALL the index of the producct that matches the target 
    idx_list = list(indices[indices == target].index)
#     print(list(indices[indices == target].index))
    
    for i in idx_list:
        # creating a Series with the similarity scores in descending order
        score_series = pd.Series(cosine_sim[i]).sort_values(ascending = False)

        # getting the indexes of the 10 most similar products
        # omit 0 bc first is itself
        top_10_indexes = list(score_series.iloc[1:11].index)

        # populating the list with the names of the best 10 matching products for each index
        for i in top_10_indexes:
            recommended_products.append(list(df['Product Name'])[i])
    
    # Sort the list so that the duplicated recommendations can appear first
    # removing duplicates that are reccommended to different indexes
    # then take only top 10
    sorted_recommends = sorted(recommended_products, key=recommended_products.count,reverse=True)
    recommended_results = list(dict.fromkeys(sorted_recommends))[:11]
    
    return recommended_results

In [18]:
recommendations("LOL Surprise IndoorOutdoor PopUp Play Tent with FoldUp Door")

['WowWee Pinkfong Baby Shark Bath Squirt Toy  4 Pack',
 'Playgo Musical Spinning Wheel',
 'Boon Water Bugs Floating Bath Toys with Net  Mint Green',
 'TOMY John Deere Learn n Pop Farmyard Friends Toy',
 'Ubbi Squeeze and Switch Silicone Mold Free Bath Toys for Toddlers and Baby',
 'The World of Eric Carle Duckie Bath Book and Squirty Toy Set',
 'Stephen Joseph Xylophone Unicorn',
 'Folkmanis Mini Sitting Frog',
 'Playgo Mystic Rainmaker Rattle',
 'Schylling Veterinarian Kit']

In [19]:
recommendations(1532)

['Fiesta Toys Lubby Cubbies Blue Berry The Penguin Plush Stuffed Toy  35 Inches',
 'Crayola Nontoxic AntiDust Chalk White 12 SticksBox 501402',
 'MightySkins Skin Compatible with DJI Phantom 3 Professional Quadcopter Drone wrap Cover Sticker Skins Solid Lavender',
 'Costume Sunglasses Clown from IT SunStaches Party Favors UV400',
 'Arcane Tinman Dragon Shield Shell Deck Box Collectible Card Blue One Size',
 'SignMission Big Cookies 72 Banner Concession Stand Food Truck Single Sided Size 24 X 72',
 'Kiddomoto Kids Bicycle Helmet in a Variety of Styles and Sizes',
 'Kaleen Rugs Lily  Liam Collection LAL0275 Grey Machine Tufted Rug 2 x 3',
 'LOL Surprise IndoorOutdoor PopUp Play Tent with FoldUp Door',
 'Forum Novelties Robot Costume for Children  Large',
 'Toy Story 4 Multicolor Spiral Party Decorations 12 Ct']